In [1]:
import pandas as pd
import numpy as np

## Functions

In [2]:
# gus = genes of unknown significance
gus = ['AKT2', 'AKT3', 'ATR', 'ATRX', 'CDKN1A', 'CDKN2B', 'CDKN2C', 'CDKN2D', 'CEBPA', 'CEP57', 'DAXX', 'EGFR', 
       'EGLN1', 'EGLN2', 'EPAS1', 'EPHB2', 'ABRAXAS1', 'FANCM', 'GPRC5A', 'HNF1A', 'HORMAD1', 'HORMAD2', 'JAK2', 
       'KIF1B', 'MAP3K1', 'MAP3K6', 'MDH2', 'MLH3', 'MMS19', 'MN1', 'MRE11', 'PALLD', 'PIK3C2G', 'PIK3R2', 'PMS1', 
       'PRF1', 'PTCH2', 'RAD50', 'RAD51', 'RECQL', 'RINT1']

# make df .xlsx conform for xlsx_df function with more columns
def conf_xlsx(df):
    dfn = df.copy()
    for i in ['consensus_sequence_IGV_region_left', 'consensus_sequence_IGV_sequence_left', 
              'consensus_sequence_IGV_region_right', 'consensus_sequence_IGV_sequence_right', 
              'INFO_IGV', 'too_low_coverage', 'splice_change']:
        dfn[i] = np.nan
    dfn['shortcut'] = dfn['#CHROM'].astype(str)+'_'+dfn['POS'].astype(str)+'_'+dfn['REF']+'_'+dfn['ALT']
    return dfn

# function to make excel tables with one variant per row
cols_cpra = ['#CHROM', 'POS', 'REF', 'ALT']
columns_df_prior = ['#CHROM', 'POS', 'REF', 'ALT', 'MaxEntScan_alt', 'MaxEntScan_diff', 'MaxEntScan_ref', 
                    'CADD_PHRED', 'SpliceAI_pred_DP_AG', 'SpliceAI_pred_DP_AL', 'SpliceAI_pred_DP_DG', 
                    'SpliceAI_pred_DP_DL', 'SpliceAI_pred_DS_AG', 'SpliceAI_pred_DS_AL', 'SpliceAI_pred_DS_DG', 
                    'SpliceAI_pred_DS_DL', 'SpliceAI_pred_SYMBOL', 'ada_score', 'rf_score',
                    'PosExonRefSeqAccession', 'PosExon_type', 'ClinVar_Pathogenicity', 
                    'consensus_sequence_IGV_region_left', 'consensus_sequence_IGV_sequence_left', 
                    'consensus_sequence_IGV_region_right', 'consensus_sequence_IGV_sequence_right', 'hgsv_new',
                    'INFO_IGV', 'too_low_coverage', 'splice_change', 'shortcut', 'ClinVar_Traits', 
                    'Splice Project Gene Priority', 'PID']
columns_df_xlsx = columns_df_prior[:-1]
columns_df_xlsx.extend(['PIDs', 'ct_PIDs'])

def xlsx_df(prio_df, name):
    lists = []
    for p,df in prio_df.groupby(by=['#CHROM', 'POS', 'REF', 'ALT']):
        df = df.reset_index(drop=True)
        list_df = [df.iloc[0][i] for i in columns_df_prior[:-1]]
        list_df.extend([list(df['PID']), len(df['PID'])])
        lists.append(list_df)
    new_df = pd.DataFrame(lists, columns=columns_df_xlsx)
    new_df['#CHROM'] = pd.Categorical(new_df['#CHROM'],
                                      categories=['1','2','3','4','5','6','7','8','9','10','11','12', '13', '14', 
                                                  '15', '16', '17', '18', '19', '20', '21', '22', 'X', 'Y'],
                                      ordered=True)
    new_df.sort_values(by=['ct_PIDs', '#CHROM', 'POS', 'REF', 'ALT'], inplace=True, ignore_index=True, 
                       ascending=[False, True, True, True, True])
    new_df.drop(axis='columns', columns='ct_PIDs', inplace=True)
    new_df.to_excel('01_xlsx_files/%s.xlsx'%name, index=False)

def xlsx_vars(prio_vars_df, name):
    merged = pd.merge(all_vars[columns_df_prior], prio_vars_df, how='inner')
    lists = []
    for p,df in merged.groupby(by=['#CHROM', 'POS', 'REF', 'ALT']):
        df = df.reset_index(drop=True)
        list_df = [df.iloc[0][i] for i in columns_df_prior[:-1]]
        list_df.extend([list(df['PID']), len(df['PID'])])
        lists.append(list_df)
    new_df = pd.DataFrame(lists, columns=columns_df_xlsx)
    new_df['#CHROM'] = pd.Categorical(new_df['#CHROM'],
                                      categories=['1','2','3','4','5','6','7','8','9','10','11','12', '13', '14', 
                                                  '15', '16', '17', '18', '19', '20', '21', '22', 'X', 'Y'],
                                      ordered=True)
    new_df.sort_values(by=['ct_PIDs', '#CHROM', 'POS', 'REF', 'ALT'], inplace=True, ignore_index=True, 
                       ascending=[False, True, True, True, True])
    new_df.drop(axis='columns', columns='ct_PIDs', inplace=True)
    new_df.to_excel('01_xlsx_files/%s.xlsx'%name, index=False)

In [3]:
# function to search for specific name in column
def search_col(df, inp):
    return [i for i in df.columns if inp in i.lower()]

# function to make spliceAI prediction scores into floats
def make_float(df):
    for i in ['AG', 'AL', 'DG', 'DL']:
        df[('SpliceAI_pred_DS_'+i)]=df[('SpliceAI_pred_DS_'+i)].astype(float)
    for i in ['ada_score', 'rf_score', 'MaxEntScan_diff', 'MaxEntScan_alt', 'MaxEntScan_ref']:
        df[i] = df[i].astype(float)
    return df

# function to merge in IGV seen variant .xlsx files with all_vars with all infos
def merge_xlsx(file_name):
    df = pd.read_excel('01_xlsx_files/'+file_name)
    merged = pd.merge(df[['#CHROM', 'POS', 'REF', 'ALT', 'consensus_sequence_IGV_region_left',
                          'consensus_sequence_IGV_sequence_left', 'consensus_sequence_IGV_region_right',
                          'consensus_sequence_IGV_sequence_right', 'exon_skipping', 'intron_retention', 
                          "3'acceptor_loss", "3'acceptor_gain", "5'donor_loss", "5'donor_gain", 'frameshift',
                          'premat_stop_cod', 'hgsv_new', 'INFO_IGV', 'too_low_coverage',
                          'splice_change', 'var_in_blood_perc', 'var_in_tumor_perc', 'counts_coverage', 
                          'perc_coverage', 'pos_last', 'pos_next', 'transc_pos_last', 'transc_pos_next', 'kind', 
                          'info_cDNA', 'more_consequences']], all_vars, on=cols_cpra, 
                      how='left').reset_index(drop=True)
    merged['shortcut'] = (merged['#CHROM'].astype(str)+'_'+merged['POS'].astype(str)+
                          '_'+merged['REF']+'_'+merged['ALT'])
    merged['ClinVar_Pathogenicity'] = merged['ClinVar_Pathogenicity'].replace(
        {'uncertain_significance':'Uncertain Significance', 'Uncertain significance':'Uncertain Significance'})
    merged['PosExon_type'] = merged['PosExon_type'].replace({'insideDonor':'insideDonorSite', 
                                                             'outsideDonor':'outsideDonorSite'})
    merged[['exon_skipping', 'intron_retention', "3'acceptor_loss", "3'acceptor_gain", "5'donor_loss", 
            "5'donor_gain"]] = merged[['exon_skipping', 'intron_retention', "3'acceptor_loss", "3'acceptor_gain", 
                                       "5'donor_loss", "5'donor_gain"]].replace({'x':True, np.nan:False})
    merged['strand'] = merged['info_cDNA'].str.split(',', expand=True)[0]
    merged = merged.drop(columns='info_cDNA')
    merged['#CHROM'] = pd.Categorical(merged['#CHROM'],
                                      categories=['1','2','3','4','5','6','7','8','9','10','11','12', '13', '14', 
                                                  '15', '16', '17', '18', '19', '20', '21', '22', 'X', 'Y'],
                                      ordered=True)
    merged.sort_values(by=['#CHROM', 'POS', 'REF', 'ALT'], inplace=True, ignore_index=True, 
                       ascending=True)
    merged = make_float(merged)
    return merged

In [4]:
# categorize variants into groups
# spliceAI score > 0.9
def splai_o09(df, coverage, splice_change):
    return len(df[((df['SpliceAI_pred_DS_AG']>0.9)|
                   (df['SpliceAI_pred_DS_AL']>0.9)|
                   (df['SpliceAI_pred_DS_DG']>0.9)|
                   (df['SpliceAI_pred_DS_DL']>0.9))&
                  (df['too_low_coverage']==coverage)&
                  (df['splice_change']==splice_change)].groupby(cols_cpra))

# ada/rf score > 0.6
def ada_rf_o06(df, coverage, splice_change):
    return len(df[(df['ada_score']>0.9)&
                  (df['rf_score']>0.9)&
                  (df['too_low_coverage']==coverage)&
                  (df['splice_change']==splice_change)].groupby(cols_cpra))

# MaxEntScan high disruption of native splice site
def MES_disr(df, coverage, splice_change):
    return len(df[(df['MaxEntScan_diff']>=0)&
                  (df['MaxEntScan_alt']<6.2)&
                  (df['too_low_coverage']==coverage)&
                  (df['splice_change']==splice_change)].groupby(cols_cpra))

# MaxEntScan high possibility of creating new splice site
def MES_new(df, coverage, splice_change):
    return len(df[(df['MaxEntScan_diff']<0)&
                  (df['MaxEntScan_alt']>8.5)&
                  (df['too_low_coverage']==coverage)&
                  (df['splice_change']==splice_change)].groupby(cols_cpra))

# position relative to exon/intron boundary less than 3 and in intron
def intron_pos(df, coverage, splice_change):
    return len(df[(df['PosExonRefSeqAccession']<3)&
                  (df['PosExon_type'].isin(['outsideAcceptorSite', 'outsideDonorSite', 'outsideDonor']))&
                  (df['too_low_coverage']==coverage)&
                  (df['splice_change']==splice_change)].groupby(cols_cpra))

# variant categorization per hot/warm/cold genes
def gene_prio(df):
    hot_lst, warm_gus_lst, warm_lst, cold_lst = ({} for i in range(4))
    for i,lst,genes in zip(['Hot (ACMG / MASTER)', 'Warm (MASTER-ACMG)', 'Warm (MASTER-ACMG)', 
                            'Cold (387-ACMG-MASTER)'], 
                           [[''], gus, list(set(df['HUGO_Symbol'][df['Splice Project Gene Priority']==
                                                                  'Warm (MASTER-ACMG)'])^set(gus)), ['']], 
                           [hot_lst, warm_gus_lst, warm_lst, cold_lst]):
        for coverage, splice_change, cat in zip(['no', 'no', 'no', 'yes', 'yes', 'yes'], 
                                            ['yes', 'maybe', 'no', 'yes', 'maybe', 'no'], 
                                            ['sufficient, yes', 'sufficient, maybe', 'sufficient, no', 
                                             'not sufficient, yes', 'not sufficient, maybe', 'not sufficient, no']):
            genes[cat] = len(df[(df['Splice Project Gene Priority']==i)&
                                (~df['HUGO_Symbol'].isin(lst))&
                                (df['too_low_coverage']==coverage)&
                                (df['splice_change']==splice_change)].groupby(cols_cpra))
    newdf = pd.DataFrame([hot_lst, warm_gus_lst, warm_lst, cold_lst], 
                        index=['hot genes', 'warm genes - GUS', 'GUS', 'cold genes'])
    newdf = newdf.reset_index(drop=False)
    newdf = newdf.rename(columns={'index':'category'})
    return newdf

# variant categorization for specific scores
def categ_vars(df):
    splAI_lst, ada_rf_lst, MES_disr_lst, MES_new_lst, intron_pos_lst = ({} for i in range(5))
    for coverage, splice_change, cat in zip(['no', 'no', 'no', 'yes', 'yes', 'yes'], 
                                            ['yes', 'maybe', 'no', 'yes', 'maybe', 'no'], 
                                            ['sufficient, yes', 'sufficient, maybe', 'sufficient, no', 
                                             'not sufficient, yes', 'not sufficient, maybe', 'not sufficient, no']):
        splAI_lst[cat] = splai_o09(df, coverage, splice_change)
        ada_rf_lst[cat] = ada_rf_o06(df, coverage, splice_change)
        MES_disr_lst[cat] = MES_disr(df, coverage, splice_change)
        MES_new_lst[cat] = MES_new(df, coverage, splice_change)
        intron_pos_lst[cat] = intron_pos(df, coverage, splice_change)
        
    newdf = pd.DataFrame([splAI_lst, ada_rf_lst, MES_disr_lst, MES_new_lst, intron_pos_lst], 
                        index=['spliceAI > 0.9', 'ada, rf score > 0.9', 'MES high disruption', 'MES high new ss', 
                               'pos. rel. to ss < 3, intron'])
    newdf = newdf.reset_index(drop=False)
    newdf = newdf.rename(columns={'index':'category'})
    return newdf

# merge variant categorization for gene priority and specific scores, create percentages/sums
def perc_cat(df):
    dfs = gene_prio(df).append(categ_vars(df), ignore_index=True)
    dfs['sum'] = (dfs['sufficient, yes']+dfs['sufficient, maybe']+dfs['sufficient, no']+
                  dfs['not sufficient, yes']+dfs['not sufficient, maybe']+dfs['not sufficient, no'])
    sum_row = {i:(dfs.iloc[0][i]+dfs.iloc[2][i]+dfs.iloc[3][i]) for i in dfs.columns[1:8]}
    dfs = dfs.append({'category':'sums'}|sum_row, ignore_index=True)
    for i in ['sufficient, yes', 'sufficient, maybe', 'sufficient, no']:
        dfs['% '+i] = dfs[i]/(dfs['sufficient, yes']+dfs['sufficient, maybe']+dfs['sufficient, no'])
    dfs['sufficient to not sufficient'] = ((dfs['sufficient, yes']+dfs['sufficient, maybe']+dfs['sufficient, no'])/
                                           (dfs['sum']))
    return dfs

In [5]:
def right_type(df):
    dfn = df.copy()
    dfn[['#CHROM', 'REF', 'ALT']] = dfn[['#CHROM', 'REF', 'ALT']].astype(str)
    dfn['POS'] = dfn['POS'].astype(int)
    return dfn

In [6]:
# dataframe with all variants
all_vars = pd.read_pickle('00_dataframes/all_annotated_vars')

all_vars['#CHROM'] = pd.Categorical(all_vars['#CHROM'],
                                    categories=['1','2','3','4','5','6','7','8','9','10','11','12', '13', '14', 
                                                '15', '16', '17', '18', '19', '20', '21', '22', 'X', 'Y'],
                                    ordered=True)
all_vars = all_vars.sort_values(by=['#CHROM', 'POS', 'REF', 'ALT'], ignore_index=True)
all_vars = right_type(all_vars)

In [7]:
igv_all = merge_xlsx('2021-08-19_all_vars_analyzed.xlsx')
igv_wo_dup = igv_all.copy().drop_duplicates(cols_cpra)
#igv_all.to_pickle('00_dataframes/2021-11-19_igv_vars')

In [8]:
import xml.etree.ElementTree as ET
root = ET.parse('01_xlsx_files/response_1637323300036.xml').getroot()

In [ ]:
chrom = t.find('chromosome').text
    pos = t.find('position').text
    ref = t.find('reference').text
    alt = t.find('alternate').text
    gene = t.find('gene').text
    gt = t.find('genotype').text
    assessment = t.find('assessment').text
    #cadd = t.find('cadd').text
    variation = t.find('variation').text
    pt = t.find('phenotype').text
    function = t.find('function').text

In [19]:
with open('01_xlsx_files/response_1637323300036.xml') as f:
    ct = 0
    ct_cl = 0
    qci_dict = {}
    for line in f.readlines():
        line = line.lstrip()
        if line.startswith('<chromosome'):
            chrom = line.split('>')[1].split('<')[0]
        elif line.startswith('<position'):
            pos = line.split('>')[1].split('<')[0]
        elif line.startswith('<reference'):
            ref = line.split('>')[1].split('<')[0]
        elif line.startswith('<alternate'):
            alt = line.split('>')[1].split('<')[0]
        elif line.startswith('<genotype'):
            gt = line.split('>')[1].split('<')[0]
        #elif line.startswith('<computedAssessment'):
            #cp_ass = line.split('="')[1].split('"')[0]
        #elif line.startswith('<assessment'):
            #ass = line.split('>')[1].split('<')[0]
        elif line.startswith('<phenotype'):
            pt = line.split('>')[1].split('<')[0]
        elif line.startswith('<classification '):
            ct += 1
            ct_cl = 0
            in_dict = {}
            qci_dict[ct] = in_dict
            
            for k,v in zip(['chromosome', 'position', 'reference', 
                            'alternate', 'genotype', 'computedAssessment', 
                            'assessment', 'phenotype'], 
                           [chrom, pos, ref, alt, gt, pt]):
                in_dict[k]=v
            
            in_dict['classified_by'] = line.split(' type="')[1].split('"')[0]
            in_dict['inheritance'] = line.split(' value="')[1].split('"')[0]
            in_dict['criteria'] = ''
            
        elif line.startswith('<criterion'):
            crit = line.split(' id="')[1].split('"')[0]+': '
            for i in line.split(' ')[2:]:
                crit += i.split('="')[1].strip('"')[0]+', '
        
        elif line.startswith('<text'):
            crit += line.split('>')[1].split('<')[0]+', '
            in_dict['criteria'] += crit
        #elif line.startswith('<rationale '):
            #crit += line.split('text="')[1].split('"')[0]
            
        '''
        elif line.startswith('<cadd'):
            cadd = line.split('>')[1].split('<')[0]
        elif line.startswith('<variation'):
            var = line.split('>')[1].split('<')[0]
        elif line.startswith('<gene'):
            gene = line.split('>')[1].split('<')[0]
        elif 
        
        elif line.startswith('<function'):
            fct = line.split('>')[1].split('<')[0]
            '''
# counts, "category"+str(ct_cl)

IndexError: list index out of range

In [12]:
qci_dict

{1: {'chromosome': '3',
  'position': '52439929',
  'reference': '0',
  'alternate': 'A',
  'genotype': 'Het',
  'computedAssessment': 'Likely Pathogenic',
  'assessment': 'Pathogenic',
  'phenotype': 'Tumor predisposition syndrome',
  'classified_by': 'QIAGEN',
  'cadd': '35.00',
  'variation': 'SNV',
  'gene': 'BAP1',
  'function': 'loss'},
 2: {'chromosome': '3',
  'position': '52440267',
  'reference': '0',
  'alternate': 'G',
  'genotype': 'Het',
  'computedAssessment': 'Pathogenic',
  'assessment': 'Likely Pathogenic',
  'phenotype': 'Tumor predisposition syndrome',
  'classified_by': 'QIAGEN',
  'cadd': '33.00',
  'variation': 'SNV',
  'gene': 'BAP1',
  'function': 'loss'},
 3: {'chromosome': '11',
  'position': '108186639',
  'reference': '0',
  'alternate': 'A',
  'genotype': 'Het',
  'computedAssessment': 'Pathogenic',
  'assessment': 'Pathogenic',
  'phenotype': 'Ataxia-telangiectasia',
  'classified_by': 'QIAGEN',
  'cadd': '33.00',
  'variation': 'SNV',
  'gene': 'ATM',
  

In [129]:
newr = []
for t in root.findall('variant'):
    for i in t.findall('classifications'):
        for s in i.findall('classification'):
            for o in s.findall('criteria'):
                for n in o.findall('criterion'):
                    print(n.find('text').text)
        #d_new = {}
        #for s in i.findall('classification'):
        #    d_new[s.find]

+ PVS1 - Null variant (nonsense, frameshift, canonical +/-1 or 2 splice sites, initiation codon, copy number loss, single or multi exon deletion) in a gene where loss of function (LOF) is a known mechanism of disease (Very Strong)
+ PM2 - Absent from controls (or at extremely low frequency if recessive) in gnomAD [In these sources of population frequency data, this variant's frequency is 0% or <= 0.001%] (Moderate)
+ PP4 - Patient's phenotype or family history is highly specific for a disease with a single genetic etiology (supporting)
+ PVS1 - Null variant (nonsense, frameshift, canonical +/-1 or 2 splice sites, initiation codon, copy number loss, single or multi exon deletion) in a gene where loss of function (LOF) is a known mechanism of disease (Very Strong)
+ PM2 - Absent from controls (or at extremely low frequency if recessive) in gnomAD [In these sources of population frequency data, this variant's frequency is 0% or <= 0.001%] (Moderate)
+ PVS1 - Null variant (nonsense, frames

In [114]:
rows = []

for t in root.findall('variant'):
    chrom = t.find('chromosome').text
    pos = t.find('position').text
    ref = t.find('reference').text
    alt = t.find('alternate').text
    gene = t.find('gene').text
    gt = t.find('genotype').text
    assessment = t.find('assessment').text
    #cadd = t.find('cadd').text
    variation = t.find('variation').text
    pt = t.find('phenotype').text
    function = t.find('function').text
    for i in t.findall('transcriptchange'):
        transc = i.find('transcript').text+':'+i.find('change').text
        ex_num = i.find('exonNumber').text
        region = i.find('region').text
    for i in t.findall('classifications'):
        class_dict = {}
        for s in i.findall('classification'):
            
            
            for l in 
        
    
    
    rows.append({'#CHROM': chrom, 'POS':pos, 'REF':ref, 'ALT':alt, 'Gene':gene, 
                 'Genotype':gt, 
                 'Assessment':assessment, 'Phenotype':pt, 'Transcript':transc, 
                 'Exon_number':ex_num, 'Region':region, 'Function':function, 
                 'Variation':variation, })

qci_df = pd.DataFrame(rows)

In [115]:
qci_df

,#CHROM,POS,REF,ALT,Gene,Genotype,Assessment,Phenotype,Transcript,Exon_number,Region,Function,Variation
0,3,52439929,C,A,BAP1,Het,Pathogenic,Tumor predisposition syndrome,NM_004656.4:c.784-1G>T,10,Splice Site,loss,SNV
1,3,52440267,A,G,BAP1,Het,Likely Pathogenic,Tumor predisposition syndrome,NM_004656.4:c.783+2T>C,9,Splice Site,loss,SNV
2,11,108186639,G,A,ATM,Het,Pathogenic,Ataxia-telangiectasia,NM_000051.4:c.6095+1G>A,41,Splice Site,loss,SNV
3,11,108224490,T,G,ATM,Het,Uncertain Significance,Ataxia-telangiectasia,NM_000051.4:c.8672-3T>G,59,Intron,loss,SNV
4,3,52443569,C,T,BAP1,Het,Pathogenic,Tumor predisposition syndrome,NM_004656.4:c.122+1G>A,3,Splice Site,loss,SNV
5,17,41234419,A,C,BRCA1,Het,Pathogenic,Hereditary breast and/or ovarian cancer,NM_007294.4:c.4357+2T>G,12,Splice Site,loss,SNV
6,17,41222939,A,G,BRCA1,Het,Pathogenic,Hereditary breast and/or ovarian cancer,NM_007294.4:c.4986+6T>C,15,Intron,loss,SNV
7,17,59934594,T,C,BRIP1,Het,Likely Pathogenic,Hereditary breast and/or ovarian cancer,NM_032043.3:c.206-2A>G,4,Splice Site,loss,SNV
8,16,68849662,CG,C,CDH1,Het,Pathogenic,Hereditary diffuse malignant gastric tumor,NM_001317184.2:c.1382+1delG,9,Splice Site,loss,Deletion
9,22,29121230,C,T,CHEK2,Het,Pathogenic,Hereditary breast cancer,NM_001005735.2:c.573+1G>A,4,Splice Site,loss,SNV


In [90]:
rows

[{'#CHROM': '3',
  'POS': '52439929',
  'REF': 'C',
  'ALT': 'A',
  'genotype': 'Het',
  'computed_assessment': 'Pathogenic',
  'phenotype': 'Tumor predisposition syndrome'},
 {'#CHROM': '3',
  'POS': '52440267',
  'REF': 'A',
  'ALT': 'G',
  'genotype': 'Het',
  'computed_assessment': 'Likely Pathogenic',
  'phenotype': 'Tumor predisposition syndrome'},
 {'#CHROM': '11',
  'POS': '108186639',
  'REF': 'G',
  'ALT': 'A',
  'genotype': 'Het',
  'computed_assessment': 'Pathogenic',
  'phenotype': 'Ataxia-telangiectasia'},
 {'#CHROM': '11',
  'POS': '108224490',
  'REF': 'T',
  'ALT': 'G',
  'genotype': 'Het',
  'computed_assessment': 'Uncertain Significance',
  'phenotype': 'Ataxia-telangiectasia'},
 {'#CHROM': '3',
  'POS': '52443569',
  'REF': 'C',
  'ALT': 'T',
  'genotype': 'Het',
  'computed_assessment': 'Pathogenic',
  'phenotype': 'Tumor predisposition syndrome'},
 {'#CHROM': '17',
  'POS': '41234419',
  'REF': 'A',
  'ALT': 'C',
  'genotype': 'Het',
  'computed_assessment': 'Path

In [84]:
from xml.dom import minidom
xmldoc = minidom.parse('01_xlsx_files/response_1637323300036.xml')
itemlist = xmldoc.getElementsByTagName('variant')
itemlist.count()
#for s in itemlist:
    #print(s.attr)

0

In [47]:


Bs_data = bs(qci, 'xml')
 
# Finding all instances of tag
# `unique`
Bs_data.find_all('unique')
 
# Using find() to extract attributes
# of the first instance of the tag
#b_name = Bs_data.find('child', {'name':'Frank'})
 
# Extracting the data stored in a
# specific attribute of the
# `child` tag
#value = b_name.get('test')

FeatureNotFound: Couldn't find a tree builder with the features you requested: xml. Do you need to install a parser library?

In [33]:
igv_all

,#CHROM,POS,REF,ALT,consensus_sequence_IGV_region_left,consensus_sequence_IGV_sequence_left,consensus_sequence_IGV_region_right,consensus_sequence_IGV_sequence_right,exon_skipping,intron_retention,...,note_hboc,add_note_hboc,splice_pred_alamut_hboc,prediction_hboc,literature_hboc,evidence_level_lit_hboc,comment_hboc,recomm_action_hboc,shortcut,strand
0,1,10384956,A,G,chr1:10384914-10384953,CCTTCTATGATCGGTTCCACTGGTTCAAACTTGTGGGGAG,chr1:10394578-10394617,GGCATTTGTTTACCTGAGCAATCTGCTGTATCCCGTGCCC,True,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1_10384956_A_G,plus
1,1,10394695,A,C,NaN,NaN,NaN,NaN,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1_10394695_A_C,NaN
2,1,10399821,A,T,NaN,NaN,NaN,NaN,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1_10399821_A_T,NaN
3,1,11740670,C,A,NaN,NaN,NaN,NaN,True,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1_11740670_C_A,minus
4,1,15767087,G,C,NaN,NaN,NaN,NaN,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1_15767087_G_C,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1458,X,47045028,A,G,NaN,NaN,NaN,NaN,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X_47045028_A_G,NaN
1459,X,53246977,C,T,NaN,NaN,NaN,NaN,False,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X_53246977_C_T,NaN
1460,X,70349523,A,G,NaN,NaN,NaN,NaN,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X_70349523_A_G,NaN
1461,X,147019617,G,A,NaN,NaN,NaN,NaN,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X_147019617_G_A,NaN


In [32]:
categ_vars(igv_wo_dup)

,category,"sufficient, yes","sufficient, maybe","sufficient, no","not sufficient, yes","not sufficient, maybe","not sufficient, no"
0,spliceAI > 0.9,50,12,19,0,14,21
1,"ada, rf score > 0.9",51,17,23,0,19,35
2,MES high disruption,63,33,77,0,22,55
3,MES high new ss,1,1,55,0,0,15
4,"pos. rel. to ss < 3, intron",37,14,19,0,11,25


In [25]:
igv_wo_dup[['shortcut', 'HUGO_Symbol']+search_col(igv_wo_dup, 'spliceai')+
           search_col(igv_wo_dup, 'clinvar')+search_col(igv_wo_dup, 'posexon')][
    ((igv_wo_dup['SpliceAI_pred_DS_AG']>0.9)|
                   (igv_wo_dup['SpliceAI_pred_DS_AL']>0.9)|
                   (igv_wo_dup['SpliceAI_pred_DS_DG']>0.9)|
                   (igv_wo_dup['SpliceAI_pred_DS_DL']>0.9))&
                  (igv_wo_dup['too_low_coverage']=='no')&
                  (igv_wo_dup['splice_change']=='no')]

,shortcut,HUGO_Symbol,SpliceAI_pred_DP_AG,SpliceAI_pred_DP_AL,SpliceAI_pred_DP_DG,SpliceAI_pred_DP_DL,SpliceAI_pred_DS_AG,SpliceAI_pred_DS_AL,SpliceAI_pred_DS_DG,SpliceAI_pred_DS_DL,SpliceAI_pred_SYMBOL,ClinVar_Pathogenicity,ClinVar_Traits,PosExonRefSeqAccession,PosExon_type
349,1_243736225_TAC_T,AKT3,-14,3,-14,3,0.00,0.00,0.64,1.00,AKT3,NaN,NaN,3,outsideDonorSite
384,2_47641558_GTA_G,MSH2,0,-26,-25,-1,0.00,0.00,0.00,1.00,MSH2,NaN,NaN,1,outsideDonorSite
415,2_225378349_TGCGCCTCTGTC_T,CUL3,-35,6,-6,6,0.31,0.94,0.00,0.00,CUL3,NaN,NaN,6,insideAcceptorSite
432,3_37053354_G_A,MLH1,50,-43,-44,-1,0.00,0.00,0.00,0.99,MLH1,NaN,NaN,1,outsideDonorSite
590,5_79968062_G_T,MSH3,8,1,8,15,0.97,1.00,0.00,0.00,MSH3,NaN,NaN,1,outsideAcceptorSite
600,5_80064822_GGTAAT_G,MSH3,0,-13,8,0,0.00,0.00,0.00,0.92,MSH3,NaN,NaN,0,insideDonorSite
654,7_95820419_A_C,SLC25A13,-28,50,-28,2,0.00,0.00,0.00,0.99,SLC25A13,NaN,NaN,2,outsideDonorSite
657,7_105190699_T_G,RINT1,1,9,-49,49,0.45,0.94,0.00,0.00,RINT1,NaN,NaN,9,outsideAcceptorSite
723,9_432158_T_A,DOCK8,2,8,27,-39,1.00,0.94,0.00,0.00,DOCK8,NaN,NaN,8,outsideAcceptorSite
765,9_137650127_C_T,COL5A1,15,-38,-2,15,0.00,0.00,0.93,0.07,COL5A1,Uncertain Significance,"Ehlers-Danlos syndrome, classic type",15,insideDonorSite


### Mutational hotspots

In [7]:
onk_merge = pd.read_excel('01_xlsx_files/2021-10-27_onkostar_merged.xlsx')
onk_merge = right_type(onk_merge)
pd.merge(onk_merge, all_vars[cols_cpra+['PID_trans', 'Control_VAF', 'Tumor_VAF', 'Control_dpALT', 
                                        'Control_dp', 'Tumor_dpALT', 'Tumor_dp', 'AF_RNA', 'AF_RNA_noSkipReads']], 
         on=['#CHROM', 'POS', 'REF', 'ALT', 'PID_trans'], how='left'
        ).to_excel('02_output_analysis/2021-10-27_onkostar_vafs.xlsx', index=False)

In [8]:
hotspots = pd.read_excel('01_xlsx_files/hotspots_v2.xls')
hotspots = hotspots[hotspots['Hugo_Symbol'].isin(onk_merge['HUGO_Symbol'].unique())].copy()

In [49]:
tp = list(hotspots['Samples'][(hotspots['Hugo_Symbol']=='TP53')&(hotspots['Genomic_Position'].str.startswith('17:7578370'))])[0].split(':')
tp_num = [t.split('|') for t in tp]
tp_num = sum([int(n) for sl in tp_num for n in sl if n.isdigit()])

### variant infos merged

In [417]:
vars1 = pd.read_excel('01_xlsx_files/vars_splice.xlsx')

vars2 = pd.read_excel('01_xlsx_files/new_vars_analysis_genes.xlsx').rename(columns={'shortcut':'shortcut_v'})
vars2[['sc0', 'sc1', 'sc2', 'sc3', 'sc4']] = vars2['shortcut_v'].str.split('_', expand=True)
vars2['shortcut'] = vars2['sc0']+'_'+vars2['sc1']+'_'+vars2['sc2']+'_'+vars2['sc3']
vars2 = vars2.drop(columns=['sc0', 'sc1', 'sc2', 'sc3', 'sc4'])

In [436]:
var_merge = pd.merge(vars1.drop(columns=['PIDs', 'kind', 'pos_last', 'pos_next']), vars2, 
                     on=['#CHROM','shortcut'], how='outer')
var_merge[['#CHROM', 'ALT', 'REF']] = var_merge[['#CHROM', 'ALT', 'REF']].astype(str)
var_merge['POS'] = var_merge['POS'].astype(int)

In [470]:
kge_hboc_wodup = kge_hboc[cols_cpra].drop_duplicates().reset_index(drop=True)
kge_hboc_wodup['ID_Person'] = [list(df['ID Person']) for i,df in kge_hboc.groupby(cols_cpra)]
kge_hboc_wodup['KGE_HBOC'] = [list(df['from']) for i,df in kge_hboc.groupby(cols_cpra)]
kge_hboc_vars = pd.merge(var_merge, kge_hboc_wodup, on=cols_cpra, how='outer')

kge_hboc_vars.to_excel('02_output_analysis/vars_facts_together.xlsx', index=False)

### 06.10.

In [18]:
igv_all = merge_xlsx('2021-08-19_all_vars_analyzed.xlsx')

In [37]:
igv_all[['shortcut', 'HUGO_Symbol', 'max_gnomAD_AF', 
         'splice_change', 'ClinVar_Pathogenicity']][(igv_all['max_gnomAD_AF'].notnull())&(igv_all['splice_change']=='yes')&
                           (igv_all['Splice Project Gene Priority']!='Cold (387-ACMG-MASTER)')&
                           (~igv_all['Splice Project Gene Priority'].isin(gus))].drop_duplicates('shortcut')

,shortcut,HUGO_Symbol,max_gnomAD_AF,splice_change,ClinVar_Pathogenicity
7,1_23219372_G_A,EPHB2,0.00021103,yes,NaN
20,1_27688743_T_C,MAP3K6,0.0037306,yes,Likely Benign
79,1_45797760_T_C,MUTYH,0.0011775,yes,Uncertain Significance
80,1_45797835_T_G,MUTYH,6.7534e-05,yes,Uncertain Significance
380,2_47601174_C_T,EPCAM,4.2257e-06,yes,Pathogenic
381,2_47607108_G_A,EPCAM,0.0021976,yes,Benign
382,2_47612302_C_G,EPCAM,0.00028683,yes,Uncertain Significance
392,2_48033789_C_T,MSH6,6.3853e-05,yes,Uncertain Significance
423,3_14190057_C_T,XPC,8.5326e-06,yes,NaN
425,3_14206927_C_A,XPC,0,yes,Likely Pathogenic


In [12]:
hboc_all = pd.read_csv('01_xlsx_files/2021-10-06_HBOC_Gepado_PML_all_variants.csv', sep=';')
hboc_all = hboc_all.dropna(axis=0, subset=['Chr', 'POS', 'REF', 'ALT']).reset_index(drop=True)
kge_all = pd.read_csv('01_xlsx_files/2021-10-06_KGE_Gepado_PML_all_variants.csv', sep=';')
kge_all = kge_all.dropna(axis=0, subset=['Chr', 'POS', 'REF', 'ALT']).reset_index(drop=True)

In [14]:
import pysam as ps
# reference sequence fasta
ref_fasta = ps.FastaFile('/mnt/g27prist/CMTD/Stephan/bcbio_installation/genomes/Hsapiens/GRCh37/seq/GRCh37.fa')
def DataCleaning(df):
    dfn = df.copy()
    # for deletion
    
    # position = position - 1
    df_del = dfn[(dfn['ALT']=='-')&(dfn['REF']!='-')].copy()
    df_del['POS'] = df_del['POS'].str.replace(' - ', '..').str.replace('-', '..')
    df_del['newPOS'] = df_del['POS'].str.split('.', expand=True)[0].astype(int)-1

    new_ref_del_l = []
    new_alt_del_l = []
    for d in range(len(df_del)):
        # position of nucleotide of first element of dataframe in which 'Alternate' not filled out
        pos_flossies = df_del.iloc[d]['newPOS']
        # reference nucleotide at specific position
        ref_refseq = ref_fasta.fetch(reference=str(df_del.iloc[d]['Chr']), start=pos_flossies-1, end=pos_flossies)
        # for alternative nucleotide reference nucleotide
        new_alt_del_l.append(ref_refseq)
        # reference nucleotide of first element of dataframe in which 'Alternate' not filled out
        ref_flossies = df_del.iloc[d]['REF']
        # new reference nucleotides for vcf annotation
        new_ref_del = ref_refseq + ref_flossies
        new_ref_del_l.append(new_ref_del)
    # add new columns
    df_del['REF_n'] = new_ref_del_l
    df_del['ALT_n'] = new_alt_del_l
    df_del.drop(columns = ['REF', 'ALT', 'POS'], inplace = True)
    df_del.rename(columns = {'REF_n':'REF', 'ALT_n':'ALT', 'newPOS':'POS', 'Chr':'#CHROM'}, inplace = True)
    
    # for insertion
    # position = position, because something was inserted at this position
    df_ins = dfn[(dfn['REF']=='-')].copy()
    df_ins['POS'] = df_ins['POS'].str.replace('-', '^')
    df_ins['newPOS'] = df_ins['POS'].str.split('^', expand=True)[0].astype(int)

    new_alt_ins_l = []
    new_ref_ins_l = []
    for i in range(len(df_ins)):
        # position of nucleotide of first element of dataframe in which 'Reference' missing
        pos_flossies = df_ins.iloc[i]['newPOS']
        # reference nucleotide at specific position
        ref_refseq = ref_fasta.fetch(reference=str(df_ins.iloc[i]['Chr']), start=pos_flossies-1, end=pos_flossies)
        # for reference nucleotide reference nucleotide
        new_ref_ins_l.append(ref_refseq)
        # reference nucleotide of first element of dataframe in which 'Reference' missing
        alt_flossies = df_ins.iloc[i]['ALT']
        # new alternate nucleotides for vcf annotation
        new_alt_ins = ref_refseq + alt_flossies
        new_alt_ins_l.append(new_alt_ins)
    # add new columns
    df_ins['ALT_n'] = new_alt_ins_l
    df_ins['REF_n'] = new_ref_ins_l
    df_ins.drop(columns = ['REF', 'ALT', 'POS'], inplace = True)
    df_ins.rename(columns = {'REF_n':'REF', 'ALT_n':'ALT', 'newPOS':'POS', 'Chr':'#CHROM'}, inplace = True)
    
    df_snv = dfn[(dfn['ALT']!='-')&(dfn['REF']!='-')].copy()
    df_snv['POS'] = df_snv['POS'].str.split('.', expand=True)[0]
    df_snv = df_snv.rename(columns={'Chr':'#CHROM'})

    dfn_new = df_snv.append([df_del, df_ins])
    dfn_new = dfn_new.sort_index()
    return dfn_new

In [15]:
kge_all_n = DataCleaning(kge_all).drop(index=[3630, 5103])
hboc_all_n = DataCleaning(hboc_all)

In [16]:
hboc_all_n[['#CHROM', 'ALT', 'REF']] = hboc_all_n[['#CHROM', 'ALT', 'REF']].astype(str)
hboc_all_n['POS'] = hboc_all_n['POS'].astype(int)
kge_all_n[['#CHROM', 'ALT', 'REF']] = kge_all_n[['#CHROM', 'ALT', 'REF']].astype(str)
kge_all_n['POS'] = kge_all_n['POS'].astype(int)

In [19]:
kge_merge = pd.merge(kge_all_n, igv_all[cols_cpra+['splice_change', 'Splice Project Gene Priority']], how='inner', 
         on=cols_cpra).drop_duplicates(cols_cpra+['ID Person'])
kge_merge = kge_merge[(kge_merge['Splice Project Gene Priority']!='Cold (387-ACMG-MASTER)')&
                      (~kge_merge['Gen'].isin(gus))&(kge_merge['splice_change']=='yes')]
kge_merge['from'] = 'KGE'

In [20]:
hboc_merge = pd.merge(hboc_all_n, igv_all[cols_cpra+['splice_change', 'Splice Project Gene Priority']], how='inner', 
         on=cols_cpra).drop_duplicates(cols_cpra+['ID Person'])
hboc_merge = hboc_merge[(hboc_merge['Splice Project Gene Priority']!='Cold (387-ACMG-MASTER)')&
                      (~hboc_merge['Gen'].isin(gus))&(hboc_merge['splice_change']=='yes')]
hboc_merge['from'] = 'HBOC'

In [21]:
kge_hboc = kge_merge.append(hboc_merge, ignore_index=True)
kge_hboc['#CHROM'] = pd.Categorical(kge_hboc['#CHROM'],
                                    categories=['1','2','3','4','5','6','7','8','9','10','11','12', '13', '14', 
                                                '15', '16', '17', '18', '19', '20', '21', '22', 'X', 'Y'],
                                    ordered=True)
kge_hboc = kge_hboc.sort_values(by=['#CHROM', 'POS', 'REF', 'ALT'], ignore_index=True)

In [333]:
kge_hboc[cols_cpra+['Gen', 'ID Person', 'Analytik ID', 'Bewertung gesamt', 'Clin Var', 'from']
        ].to_excel('02_output_analysis/kge_hboc_spl_vars.xlsx', index=False)

### 01.09.

In [6]:
igv_all = merge_xlsx('2021-08-19_all_vars_analyzed.xlsx')

In [45]:
newn = igv_all[cols_cpra+['HUGO_Symbol','PID_trans']][
    (igv_all['Splice Project Gene Priority']!='Cold (387-ACMG-MASTER)')&
    (~igv_all['HUGO_Symbol'].isin(gus))&
    (igv_all['splice_change']=='yes')].copy().reset_index(drop=True)
newn = newn.drop(index=[17,18,19,20,21,22,23,24,25,26,27,28,29,30,32,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,
                 52,53,54,86,90,71,72]).reset_index(drop=True)

In [47]:
#newn.to_excel('02_output_analysis/PIDs_patient_data.xlsx', index=False)

In [7]:
wo_dups = igv_all.drop_duplicates(cols_cpra).copy().reset_index()

In [8]:
wo_dups[cols_cpra+['splice_change', 'too_low_coverage', 'PosExonRefSeqAccession', 'PosExon_type',
                   'ClinVar_Pathogenicity', 'MaxEntScan_alt',
                   'MaxEntScan_diff', 'MaxEntScan_ref', 'SpliceAI_pred_DP_AG', 'SpliceAI_pred_DP_AL',
                   'SpliceAI_pred_DP_DG', 'SpliceAI_pred_DP_DL', 'SpliceAI_pred_DS_AG', 'SpliceAI_pred_DS_AL',
                    'SpliceAI_pred_DS_DG', 'SpliceAI_pred_DS_DL', 'ada_score', 'rf_score']
       ][(wo_dups['splice_change']=='yes')&
         ((wo_dups['SpliceAI_pred_DS_AG']<0.9)&(wo_dups['SpliceAI_pred_DS_AL']<0.9)&
          (wo_dups['SpliceAI_pred_DS_DG']<0.9)&(wo_dups['SpliceAI_pred_DS_DL']<0.9))&
         (wo_dups['ada_score']<0.9)&(wo_dups['rf_score']<0.9)&
         (((wo_dups['MaxEntScan_diff']>=0)&(wo_dups['MaxEntScan_alt']<8.5))|
          ((wo_dups['MaxEntScan_diff']<0)&(wo_dups['MaxEntScan_alt']>6.2)))]

,#CHROM,POS,REF,ALT,splice_change,too_low_coverage,PosExonRefSeqAccession,PosExon_type,ClinVar_Pathogenicity,MaxEntScan_alt,...,SpliceAI_pred_DP_AG,SpliceAI_pred_DP_AL,SpliceAI_pred_DP_DG,SpliceAI_pred_DP_DL,SpliceAI_pred_DS_AG,SpliceAI_pred_DS_AL,SpliceAI_pred_DS_DG,SpliceAI_pred_DS_DL,ada_score,rf_score
103,2,48033789,C,T,yes,no,1,insideDonorSite,Uncertain Significance,4.269,...,1,-24,-43,1,0.00,0.00,0.0,0.07,0.128779,0.324
550,22,24175755,G,A,yes,no,4,outsideAcceptorSite,Uncertain Significance,9.546,...,24,4,4,24,0.66,0.02,0.0,0.00,0.000020,0.002


In [8]:
# vcf file for QCI (09.09.2021)
import pysam as ps

In [9]:
igv_hot_warm = igv_all[cols_cpra][(igv_all['splice_change']=='yes')&
                                  (igv_all['Splice Project Gene Priority']!='Cold (387-ACMG-MASTER)')&
                                  (~igv_all['HUGO_Symbol'].isin(gus))].drop_duplicates()

vcf_header = ps.VariantHeader()
for i in igv_hot_warm['#CHROM'].unique():
    vcf_header.add_meta('contig', items = [('ID', i)])
vcf_header.add_meta('reference', 
                    value='/AnnotierungsProzesse/Referenzdaten/Genome/Human/hg19/hg19_Ensembl/hg19 Ensembl')

vcf_out = ps.VariantFile('01_xlsx_files/hot_warm_splice.txt', 'w', header=vcf_header)

for c,p,r,a in zip(igv_hot_warm['#CHROM'], igv_hot_warm['POS'], igv_hot_warm['REF'], igv_hot_warm['ALT']):
    rec = vcf_out.new_record()
    rec.chrom = str(c)
    rec.start = p-1
    rec.stop = p
    rec.ref = r
    rec.alts = a
    vcf_out.write(rec)
vcf_out.close()

In [10]:
# ALT sequences were comma-separated --> 
with open('01_xlsx_files/hot_warm_splice.vcf', 'w') as w:
    with open('01_xlsx_files/hot_warm_splice.txt') as o:
        for line in o.readlines():
            if line.startswith('#'):
                w.write(line)
            else:
                line = line.replace(',','')
                w.write(line)

In [63]:
igv_all[['#CHROM', 'pos_last', 'pos_next', 'kind', 'shortcut', 'strand', 'HUGO_Symbol'
        ]][igv_all['pos_last'].notnull()].drop_duplicates('shortcut').to_excel('03_for_cDNA/pre_for_cDNA.xlsx', 
                                                                               index=False)

In [15]:
'''
igv_all[['#CHROM', 'POS', 'REF', 'ALT', 'HGVSg', 'HUGO_Symbol', 'shortcut', 'frameshift', 'premat_stop_cod', 'hgsv_new', 'INFO_IGV', 'too_low_coverage',
         'splice_change', 'kind_hboc', 'classification_hboc', 'aquisition_hboc', 'taskforce_review_hboc',
         'date_taskforce_hboc', 'note_hboc', 'add_note_hboc', 'splice_pred_alamut_hboc', 'prediction_hboc',
         'literature_hboc', 'evidence_level_lit_hboc', 'comment_hboc', 'recomm_action_hboc']][
    (igv_all['splice_change']=='yes')&(igv_all['kind_hboc'].notnull())&(igv_all['too_low_coverage']=='no')
].drop_duplicates(cols_cpra).to_excel('02_output_analysis/hboc_splice_change.xlsx')
'''

In [16]:
'''
igv_all[['#CHROM', 'POS', 'REF', 'ALT', 'HGVSg', 'HUGO_Symbol', 'shortcut', 'frameshift', 'premat_stop_cod', 'hgsv_new', 'INFO_IGV', 'too_low_coverage',
         'splice_change', 'kind_hboc', 'classification_hboc', 'aquisition_hboc', 'taskforce_review_hboc', 
         'date_taskforce_hboc', 'note_hboc', 'add_note_hboc', 'splice_pred_alamut_hboc', 'prediction_hboc', 
         'literature_hboc', 'evidence_level_lit_hboc', 'comment_hboc', 'recomm_action_hboc']][
    (igv_all['splice_change']=='no')&(igv_all['kind_hboc'].notnull())&(igv_all['too_low_coverage']=='no')
].drop_duplicates(cols_cpra).to_excel('02_output_analysis/hboc_no_splice_change.xlsx')
'''

### 30.08.

In [11]:
igv_all = merge_xlsx('2021-08-19_all_vars_analyzed.xlsx')
nct_vars = pd.read_pickle('00_dataframes/nct_gepado_master_vars')

igv_nct = pd.merge(nct_vars[cols_cpra].drop_duplicates(), igv_all, how='inner')

igv_outcome = perc_cat(igv_all)
transp_outcome = igv_outcome.transpose(copy=True).reset_index().drop(index=[7,8,9,10,11])
transp_outcome = transp_outcome.rename(columns={i:cat for i,cat in zip(transp_outcome.columns, 
                                                                       list(transp_outcome.loc[0]))}).drop(index=0)

In [26]:
igv_nct[cols_cpra+['SpliceAI_pred_DP_AG', 'SpliceAI_pred_DP_AL', 'SpliceAI_pred_DP_DG', 
                    'SpliceAI_pred_DP_DL', 'SpliceAI_pred_DS_AG', 'SpliceAI_pred_DS_AL', 'SpliceAI_pred_DS_DG', 
                    'SpliceAI_pred_DS_DL', 'SpliceAI_pred_SYMBOL', 'ada_score', 'rf_score',
                    'PosExonRefSeqAccession', 'PosExon_type', 'ClinVar_Pathogenicity', 'INFO_IGV', 'shortcut']][
    (igv_nct['Splice Project Gene Priority']!='Cold (387-ACMG-MASTER)')&(~igv_nct['HUGO_Symbol'].isin(gus))&
    (igv_nct['splice_change']=='yes')&(igv_nct['too_low_coverage']=='no')].drop_duplicates()

,#CHROM,POS,REF,ALT,SpliceAI_pred_DP_AG,SpliceAI_pred_DP_AL,SpliceAI_pred_DP_DG,SpliceAI_pred_DP_DL,SpliceAI_pred_DS_AG,SpliceAI_pred_DS_AL,SpliceAI_pred_DS_DG,SpliceAI_pred_DS_DL,SpliceAI_pred_SYMBOL,ada_score,rf_score,PosExonRefSeqAccession,PosExon_type,ClinVar_Pathogenicity,INFO_IGV,shortcut
0,8,30916058,A,G,29,-28,29,1,0.00,0.00,0.06,0.00,WRN,0.991457,0.700,1,insideDonorSite,Uncertain Significance,new 5'ss in intron (like spliceAI said) in 2LJ...,8_30916058_A_G
35,17,7579699,C,T,-11,22,-11,1,0.00,0.00,0.25,1.00,TP53,0.999987,0.940,22,insideAcceptorSite,NaN,exon skipping,17_7579699_C_T
42,2,48033789,C,T,1,-24,-43,1,0.00,0.00,0.00,0.07,MSH6,0.128779,0.324,1,insideDonorSite,Uncertain Significance,"some exon skipping, intron retention, in other...",2_48033789_C_T
49,9,100437871,T,C,-17,-2,-15,-10,0.09,1.00,0.00,0.00,XPA,0.999835,0.726,2,outsideAcceptorSite,NaN,"acceptor loss, new 3'ss in exon, frameshift",9_100437871_T_C
57,17,59934594,T,C,-11,-2,-2,-21,0.90,0.99,0.00,0.00,BRIP1,0.999963,0.830,2,outsideAcceptorSite,Likely Pathogenic,"new ss in exon, 9 bases of exon skipped",17_59934594_T_C
61,1,17345454,C,G,-14,-1,38,-18,0.30,0.98,0.00,0.00,SDHB,0.999986,0.938,1,outsideAcceptorSite,NaN,"new 3'ss in exon, part of exon skipped, frames...",1_17345454_C_G
65,16,68849662,CG,C,6,-31,6,0,0.00,0.00,0.88,1.00,CDH1,NaN,NaN,0,insideDonorSite,Pathogenic,"donor loss, intron retention",16_68849662_CG_C
72,2,47601174,C,T,13,2,-2,13,0.00,0.00,0.95,0.28,EPCAM,NaN,NaN,13,insideDonorSite,Pathogenic,"about 50 % new 5' ss in exon, frameshift, spli...",2_47601174_C_T
73,3,14212051,C,G,-13,-1,-3,-11,0.86,1.00,0.00,0.00,XPC,0.999946,0.918,1,outsideAcceptorSite,NaN,"intron retention, acceptor loss, new 3' ss in ...",3_14212051_C_G
77,12,133254296,C,A,-3,-1,-3,-1,0.88,0.00,0.00,0.00,POLE,NaN,NaN,9,insideAcceptorSite,Uncertain Significance,"new 3'Ss in exon (as spliceAI said), in frame,...",12_133254296_C_A


In [64]:
#[cols_cpra+['SpliceAI_pred_DP_AG', 'SpliceAI_pred_DP_AL', 'SpliceAI_pred_DP_DG', 
                    #'SpliceAI_pred_DP_DL', 'SpliceAI_pred_DS_AG', 'SpliceAI_pred_DS_AL', 'SpliceAI_pred_DS_DG', 
                    #'SpliceAI_pred_DS_DL', 'SpliceAI_pred_SYMBOL', 'ada_score', 'rf_score',
                    #'PosExonRefSeqAccession', 'PosExon_type', 'ClinVar_Pathogenicity', 'INFO_IGV']]
len(igv_all[(igv_all['too_low_coverage']=='no')&(igv_all['splice_change']=='no')&
    (((igv_all['PosExonRefSeqAccession']<3)&
     (igv_all['PosExon_type'].isin(['outsideAcceptorSite', 'outsideDonorSite'])))|
    ((igv_all['PosExonRefSeqAccession']<2)&
     (igv_all['PosExon_type'].isin(['insideAcceptorSite', 'insideDonorSite']))))
    ].groupby(cols_cpra))

81

In [78]:
igv_all[cols_cpra+['SpliceAI_pred_DP_AG', 'SpliceAI_pred_DP_AL', 'SpliceAI_pred_DP_DG', 
                    'SpliceAI_pred_DP_DL', 'SpliceAI_pred_DS_AG', 'SpliceAI_pred_DS_AL', 'SpliceAI_pred_DS_DG', 
                    'SpliceAI_pred_DS_DL', 'SpliceAI_pred_SYMBOL', 'ada_score', 'rf_score',
                    'PosExonRefSeqAccession', 'PosExon_type', 'ClinVar_Pathogenicity', 'INFO_IGV']][
    ((igv_all['SpliceAI_pred_DS_AG']<0.9)&(igv_all['SpliceAI_pred_DS_AL']<0.9)&
             (igv_all['SpliceAI_pred_DS_DG']<0.9)&(igv_all['SpliceAI_pred_DS_DL']<0.9))&
            (igv_all['ada_score']<0.9)&(igv_all['rf_score']<0.9)&
            #(((igv_all['MaxEntScan_diff']<0)&(igv_all['MaxEntScan_alt']>6.2))|
             #((igv_all['MaxEntScan_diff']>0)&(igv_all['MaxEntScan_alt']<8.5)))&
            (igv_all['too_low_coverage']=='no')&(igv_all['splice_change']=='yes')]

,#CHROM,POS,REF,ALT,SpliceAI_pred_DP_AG,SpliceAI_pred_DP_AL,SpliceAI_pred_DP_DG,SpliceAI_pred_DP_DL,SpliceAI_pred_DS_AG,SpliceAI_pred_DS_AL,SpliceAI_pred_DS_DG,SpliceAI_pred_DS_DL,SpliceAI_pred_SYMBOL,ada_score,rf_score,PosExonRefSeqAccession,PosExon_type,ClinVar_Pathogenicity,INFO_IGV
392,2,48033789,C,T,1,-24,-43,1,0.00,0.00,0.00,0.07,MSH6,0.128779,0.324,1,insideDonorSite,Uncertain Significance,"some exon skipping, intron retention, in other..."
1184,17,74469327,G,A,6,27,2,7,0.00,0.00,0.64,0.01,RHBDF2,0.000024,0.000,7,outsideDonorSite,Benign,"new 5'ss in intron (as spliceAI said), intron ..."
1442,22,24175755,G,A,24,4,4,24,0.66,0.02,0.00,0.00,SMARCB1,0.000020,0.002,4,outsideAcceptorSite,Uncertain Significance,"new 3'ss in exon (as spliceAI said) in 8Q2LVZ,..."
1443,22,24175755,G,A,24,4,4,24,0.66,0.02,0.00,0.00,SMARCB1,0.000020,0.002,4,outsideAcceptorSite,Uncertain Significance,"new 3'ss in exon (as spliceAI said) in 8Q2LVZ,..."


In [ ]:
#igv_outcome.to_excel('02_output_analysis/2021-08-19_outcome_igv.xlsx', index=False)
#igv_outcome.to_pickle('00_dataframes/2021-08-19_igv_perc_table')
#transp_outcome.to_pickle('00_dataframes/transp_igv_outcome')
#igv_all.to_pickle('00_dataframes/2021-08-19_igv_outcome')

In [76]:
#igv_outcome.to_excel('02_output_analysis/2021-08-19_outcome_igv.xlsx')

### 26.08.

In [ ]:
igv_all = merge_xlsx('2021-08-18_all_vars_analyzed.xlsx')
igv_outcome = perc_cat(igv_all)
hboc_vars = pd.read_pickle('../04_annotations/00_dataframes/hboc_vars')

In [ ]:
search_col(igv_all, 'clinv')

In [ ]:
hboc_igv = pd.merge(igv_all[['#CHROM', 'POS', 'REF', 'ALT', 'PID', 'splice_change', 'ClinVar_Pathogenicity']], 
                    hboc_vars[['#CHROM', 'POS', 'REF', 'ALT', 'ID_hboc']], 
                    on=cols_cpra,
                    how='inner').drop_duplicates()

In [ ]:
hboc_igv[hboc_igv['splice_change']=='yes'].drop_duplicates(subset=cols_cpra)

In [ ]:
len(igv_all[cols_cpra][igv_all['splice_change']=='yes'])

### 24.08.

In [ ]:
igv_all = merge_xlsx('2021-08-18_all_vars_analyzed.xlsx')
igv_outcome = perc_cat(igv_all)

In [ ]:
search_col(igv_all,'hgv')

In [ ]:
igv_all['ClinVar_Pathogenicity'].unique()

In [ ]:
asdf = igv_all[(igv_all['splice_change']=='yes')
               &(igv_all['ClinVar_Pathogenicity'].isin(['Uncertain Significance', np.nan]))
               &(igv_all['Splice Project Gene Priority']!='Cold (387-ACMG-MASTER)')
               &(~igv_all['HUGO_Symbol'].isin(gus))
               &(((igv_all['PosExonRefSeqAccession']>2)
                  &(igv_all['PosExon_type'].isin(['outsideDonorSite', 'outsideAcceptorSite'])))
                 |((igv_all['PosExonRefSeqAccession']>0)
                   &(igv_all['PosExon_type'].isin(['insideDonorSite', 'insideAcceptorSite']))))
       ]#.drop_duplicates(subset=cols_cpra)
len(asdf[['shortcut','splice_change', 'too_low_coverage', 'PosExonRefSeqAccession', 'PosExon_type', 'INFO_IGV']])

In [ ]:
#xlsx_df(asdf, '2021-08-24_hot_warm_uncertain_unusual')

### 18.08.

In [ ]:
# previously 4 duplicates -> newest version, more info...
# newest file (2021-08-18_all_vars_analyzed.xlsx) no duplicates anymore
igv_all = merge_xlsx('2021-08-18_all_vars_analyzed.xlsx')
igv_outcome = perc_cat(igv_all)
#igv_outcome.to_excel('02_output_analysis/2021-08-18_outcome_igv.xlsx', index=False)

In [ ]:
#xlsx_df(igv_all, '2021-08-19_all_vars_analyzed')

In [ ]:
#for coverage, splice_change, cat in zip(['no', 'no', 'no', 'yes', 'yes', 'yes'], 
#                                        ['yes', 'maybe', 'no', 'yes', 'maybe', 'no'], 
#                                        ['sufficient, yes', 'sufficient, maybe', 'sufficient, no', 
#                                         'not sufficient, yes', 'not sufficient, maybe', 'not sufficient, no']):
#    print(cat, len(igv_all[((igv_all['SpliceAI_pred_DS_AG']>0.5)|
#                   (igv_all['SpliceAI_pred_DS_AL']>0.5)|
#                   (igv_all['SpliceAI_pred_DS_DG']>0.5)|
#                   (igv_all['SpliceAI_pred_DS_DL']>0.5))&
#                  (igv_all['too_low_coverage']==coverage)&
#                  (igv_all['splice_change']==splice_change)].groupby(cols_cpra)))

In [ ]:
transp_outcome = igv_outcome.transpose(copy=True).reset_index().drop(index=[7,8,9,10,11])
transp_outcome = transp_outcome.rename(columns={i:cat for i,cat in zip(transp_outcome.columns, 
                                                                       list(transp_outcome.loc[0]))}).drop(index=0)

In [ ]:
#transp_outcome.to_pickle('00_dataframes/transp_igv_outcome')
#igv_all.to_pickle('00_dataframes/2021-08-19_igv_outcome')
#igv_outcome.to_pickle('00_dataframes/2021-08-19_igv_perc_table')

### 12.08. new

In [ ]:
# newest version up to 2021-08-12 13:30 2021-08-12_all_vars_analyzed.xlsx
new_vars = merge_xlsx('2021-08-12_all_vars_analyzed.xlsx')
new_vars = make_float(new_vars)

In [ ]:
new_vars.to_pickle('00_dataframes/igv_vars')

In [ ]:
#len(new_vars[((new_vars['SpliceAI_pred_DS_AG']>0.9)|
#              (new_vars['SpliceAI_pred_DS_AL']>0.9)|
#              (new_vars['SpliceAI_pred_DS_DG']>0.9)|
#              (new_vars['SpliceAI_pred_DS_DL']>0.9))&
#             (new_vars['too_low_coverage']=='yes')&
#             (new_vars['splice_change']=='no')].groupby(
#    cols_cpra))

In [ ]:
# interesting variants for further analysis on clinical patient basis
int_vars = new_vars[(new_vars['Splice Project Gene Priority']==
             'Hot (ACMG / MASTER)')&
             (new_vars['too_low_coverage']=='no')&
             (new_vars['splice_change']=='yes')]

int_vars[['#CHROM', 'POS', 'REF', 'ALT', 'MaxEntScan_alt', 'MaxEntScan_diff', 'MaxEntScan_ref', 
                    'CADD_PHRED', 'SpliceAI_pred_DP_AG', 'SpliceAI_pred_DP_AL', 'SpliceAI_pred_DP_DG', 
                    'SpliceAI_pred_DP_DL', 'SpliceAI_pred_DS_AG', 'SpliceAI_pred_DS_AL', 'SpliceAI_pred_DS_DG', 
                    'SpliceAI_pred_DS_DL', 'SpliceAI_pred_SYMBOL', 'ada_score', 'rf_score',
                    'PosExonRefSeqAccession', 'PosExon_type', 'ClinVar_Pathogenicity', 
                    'consensus_sequence_IGV_region_left', 'consensus_sequence_IGV_sequence_left', 
                    'consensus_sequence_IGV_region_right', 'consensus_sequence_IGV_sequence_right', 'INFO_IGV', 
                    'too_low_coverage', 'splice_change', 'shortcut', 'ClinVar_Traits', 'PID']].to_excel('01_xlsx_files/int_vars_pats_hot.xlsx')

### 12.08.2021

In [ ]:
# newest version up to 2021-08-12 10:00 2021-08-11_all_vars_analyzed.xlsx
#new_vars = merge_xlsx('2021-08-11_all_vars_analyzed.xlsx')

In [ ]:
warm_vars = make_float(all_vars[(all_vars['Splice Project Gene Priority']=='Warm (MASTER-ACMG)')&
                                (~all_vars['HUGO_Symbol'].isin(gus))&
                                (~all_vars['HGVSg'].isin(new_vars['HGVSg']))].copy())
warm_splai = warm_vars[(warm_vars['SpliceAI_pred_DS_AG']>0.4)|(warm_vars['SpliceAI_pred_DS_AL']>0.4)|
                       (warm_vars['SpliceAI_pred_DS_DG']>0.4)|(warm_vars['SpliceAI_pred_DS_DL']>0.4)].copy()
warm_dbsc = warm_vars[(warm_vars['ada_score']>0.6)&(warm_vars['rf_score']>0.6)&
                      (~warm_vars['HGVSg'].isin(warm_splai['HGVSg']))]

In [ ]:
#xlsx_df(conf_xlsx(warm_splai), 'warm_splai_o04')
#xlsx_df(conf_xlsx(warm_dbsc), 'warm_dbsc_o06')

### 11.08. new

In [ ]:
df_xlsx = pd.read_excel('01_xlsx_files/2021-08-11_variants_seen_no_dup_new.xlsx')

In [ ]:
# duplicates for specific variants
# 434 to 428 variants
grpd_xlsx = df_xlsx.groupby(by=cols_cpra)
grpd_lst = [(i,len(df)) for i,df in grpd_xlsx if len(df)>1]

In [ ]:
# to search in each group for best keep
#grpd_xlsx.get_group(grpd_lst[5][0])[['#CHROM', 'POS', 'REF', 'ALT', 'INFO_IGV','too_low_coverage','splice_change', 
                                     #'PIDs', 'consensus_sequence_IGV_region_left', 
                                     #'consensus_sequence_IGV_sequence_left','consensus_sequence_IGV_region_right',
                                     #'consensus_sequence_IGV_sequence_right']]

In [ ]:
df_xlsx = df_xlsx.drop(index=[401, 403, 387, 409, 430, 399])

In [ ]:
vars_df = pd.merge(df_xlsx[['#CHROM', 'POS', 'REF', 'ALT', 'consensus_sequence_IGV_region_left',
                            'consensus_sequence_IGV_sequence_left', 'consensus_sequence_IGV_region_right',
                            'consensus_sequence_IGV_sequence_right', 'INFO_IGV', 'too_low_coverage',
                            'splice_change']], all_vars, on=cols_cpra, how='left').reset_index(drop=True)

In [ ]:
vars_df['shortcut'] = vars_df['#CHROM']+'_'+vars_df['POS'].astype(str)+'_'+vars_df['REF']+'_'+vars_df['ALT']

In [ ]:
#xlsx_df(vars_df, '2021-08-11_all_vars_analyzed')

In [ ]:
# filter out cold and gus variants
rel_vars = vars_df[(vars_df['Splice Project Gene Priority']!='Cold (387-ACMG-MASTER)')&
                   (~vars_df['HUGO_Symbol'].isin(gus))].copy()

In [ ]:
len(all_vars[(all_vars['Splice Project Gene Priority']!='Cold (387-ACMG-MASTER)')&
             (~all_vars['HUGO_Symbol'].isin(gus))]['HUGO_Symbol'].unique())

In [ ]:
rel_vars[['#CHROM', 'POS', 'ALT', 'REF', 'PID', 'ClinVar_Pathogenicity', 'HGVSc', 'PosExonRefSeqAccession', 'PosExon_type', 
         'HUGO_Symbol', 'shortcut']][rel_vars['splice_change']=='yes'].drop_duplicates(cols_cpra)

In [ ]:
vars_df[['#CHROM', 'POS', 'ALT', 'REF', 'PID', 'ClinVar_Pathogenicity', 'HGVSc', 'PosExonRefSeqAccession', 'PosExon_type', 
         'HUGO_Symbol', 'shortcut']][(vars_df['Splice Project Gene Priority']=='Hot (ACMG / MASTER)')&(vars_df['splice_change']=='yes')]

### 11.08.2021

In [ ]:
df_xlsx = pd.read_excel('01_xlsx_files/2021-08-11_variants_seen_no_dup.xlsx')
all_vars = pd.read_pickle('00_dataframes/all_vars')

all_vars['#CHROM'] = pd.Categorical(all_vars['#CHROM'],
                                    categories=['1','2','3','4','5','6','7','8','9','10','11','12', '13', '14', 
                                                '15', '16', '17', '18', '19', '20', '21', '22', 'X', 'Y'],
                                    ordered=True)
all_vars = all_vars.sort_values(by=['#CHROM', 'POS', 'REF', 'ALT'], ignore_index=True)

In [ ]:
vars_df = pd.merge(df_xlsx[['#CHROM', 'POS', 'REF', 'ALT', 'consensus_sequence_IGV_region_left',
                            'consensus_sequence_IGV_sequence_left', 'consensus_sequence_IGV_region_right',
                            'consensus_sequence_IGV_sequence_right', 'INFO_IGV', 'too_low_coverage',
                            'splice_change', 'shortcut']], all_vars, on=cols_cpra, how='left')

In [ ]:
hot_vars = make_float(all_vars[(all_vars['Splice Project Gene Priority']=='Hot (ACMG / MASTER)')&
                               (~all_vars['HGVSg'].isin(vars_df['HGVSg']))&
                               (all_vars['ada_score'].notnull())].copy())
hot_vars[['ada_score', 'rf_score']] = hot_vars[['ada_score', 'rf_score']].astype(float)
hot_ada_rf = hot_vars[(hot_vars['ada_score']>0.1)&(hot_vars['rf_score']>0.1)].copy()

for i in ['consensus_sequence_IGV_region_left', 'consensus_sequence_IGV_sequence_left',  
          'consensus_sequence_IGV_region_right', 'consensus_sequence_IGV_sequence_right', 
          'INFO_IGV', 'too_low_coverage', 'splice_change', 'shortcut']:
    hot_ada_rf[i] = np.nan

In [ ]:
#xlsx_df(hot_ada_rf, 'hot_genes_adarf_o0')

In [ ]:
hot_pos = hot_vars[(hot_vars['PosExonRefSeqAccession']<6)&(~hot_vars['HGVSg'].isin(hot_ada_rf['HGVSg']))].copy()
for i in ['consensus_sequence_IGV_region_left', 'consensus_sequence_IGV_sequence_left',  
          'consensus_sequence_IGV_region_right', 'consensus_sequence_IGV_sequence_right', 
          'INFO_IGV', 'too_low_coverage', 'splice_change', 'shortcut']:
    hot_pos[i] = np.nan

In [ ]:
#xlsx_df(hot_pos, 'hot_genes_pos_l6')

### 10.08.2021

In [ ]:
df_xlsx = pd.read_excel('01_xlsx_files/2021-08-10_variants_seen_no_dup.xlsx')
all_vars = pd.read_pickle('00_dataframes/all_vars')

all_vars['#CHROM'] = pd.Categorical(all_vars['#CHROM'],
                                    categories=['1','2','3','4','5','6','7','8','9','10','11','12', '13', '14', 
                                                '15', '16', '17', '18', '19', '20', '21', '22', 'X', 'Y'],
                                    ordered=True)
all_vars = all_vars.sort_values(by=['#CHROM', 'POS', 'REF', 'ALT'], ignore_index=True)

In [ ]:
vars_df = pd.merge(df_xlsx[['#CHROM', 'POS', 'REF', 'ALT', 'consensus_sequence_IGV_region_left',
                            'consensus_sequence_IGV_sequence_left', 'consensus_sequence_IGV_region_right',
                            'consensus_sequence_IGV_sequence_right', 'INFO_IGV', 'too_low_coverage',
                            'splice_change', 'shortcut']], all_vars, on=cols_cpra, how='left')

In [ ]:
hot_vars = make_float(all_vars[(all_vars['Splice Project Gene Priority']=='Hot (ACMG / MASTER)')&
                               (~all_vars['HGVSg'].isin(vars_df['HGVSg']))].copy())
hot_splai = hot_vars[(hot_vars['SpliceAI_pred_DS_AG']>0.1)|(hot_vars['SpliceAI_pred_DS_AL']>0.1)|
                     (hot_vars['SpliceAI_pred_DS_DG']>0.1)|(hot_vars['SpliceAI_pred_DS_DL']>0.1)].copy()

for i in ['consensus_sequence_IGV_region_left', 'consensus_sequence_IGV_sequence_left',  
          'consensus_sequence_IGV_region_right', 'consensus_sequence_IGV_sequence_right', 
          'INFO_IGV', 'too_low_coverage', 'splice_change', 'shortcut']:
    hot_splai[i] = np.nan

In [ ]:
#xlsx_df(hot_splai, 'hot_genes_splai_o0')

### 09.08.2021

In [ ]:
df_xlsx = pd.read_excel('01_xlsx_files/2021-08-07_variants_seen_no_dup.xlsx')
all_vars = pd.read_pickle('00_dataframes/all_vars')
red_xlsx = pd.read_excel('01_xlsx_files/2021-08-09_redund_vars.xlsx')

all_vars['#CHROM'] = pd.Categorical(all_vars['#CHROM'],
                                    categories=['1','2','3','4','5','6','7','8','9','10','11','12', '13', '14', 
                                                '15', '16', '17', '18', '19', '20', '21', '22', 'X', 'Y'],
                                    ordered=True)
all_vars = all_vars.sort_values(by=['#CHROM', 'POS', 'REF', 'ALT'], ignore_index=True)


# no duplicates in df_xlsx --> merge with all_vars

vars_df = pd.merge(df_xlsx[['#CHROM', 'POS', 'REF', 'ALT', 'consensus_sequence_IGV_region_left',
                            'consensus_sequence_IGV_sequence_left', 'consensus_sequence_IGV_region_right',
                            'consensus_sequence_IGV_sequence_right', 'INFO_IGV', 'too_low_coverage',
                            'splice_change', 'shortcut']], all_vars, on=cols_cpra, how='left')

In [ ]:
# for variants in more than one patient
grpd_all_vars = vars_df.groupby(cols_cpra)
grpd_lst = [(i,len(df)) for i,df in grpd_all_vars if len(df)>1]

redund_df = pd.DataFrame()
for i, l in grpd_lst:
    redund_df = redund_df.append(grpd_all_vars.get_group(i))

In [ ]:
#xlsx_df(redund_df, '2021-08-09_redund_vars')

In [ ]:
# combine new characterized redundant variants with other variants
new_df_xlsx = df_xlsx[~df_xlsx['shortcut'].isin(redund_df['shortcut'])]
new_df_xlsx = new_df_xlsx.append(red_xlsx)
new_df_xlsx = new_df_xlsx.reset_index(drop=True)
new_vars_df = pd.merge(new_df_xlsx[['#CHROM', 'POS', 'REF', 'ALT', 'consensus_sequence_IGV_region_left',
                            'consensus_sequence_IGV_sequence_left', 'consensus_sequence_IGV_region_right',
                            'consensus_sequence_IGV_sequence_right', 'INFO_IGV', 'too_low_coverage',
                            'splice_change', 'shortcut']], all_vars, on=cols_cpra, how='left')
new_onl_vars = new_vars_df.drop_duplicates(cols_cpra).reset_index(drop=True)

#### IGV splice effect seen

In [ ]:
# splice effect seen, safe (not too low coverage, only if sure (splice_change == yes)) --> 88 variants
# if splice_change == maybe --> 38 variants, if splice_change == no --> 136 variants
# if too low coverage and splice_change == no --> 75 variants, splice_change == maybe --> 27 variants

#len(new_onl_vars[(new_onl_vars['too_low_coverage']=='yes')&
                 #(new_onl_vars['splice_change']=='yes')].reset_index(drop=True))

new_spl_yes = new_onl_vars[(new_onl_vars['too_low_coverage']=='no')&
                           (new_onl_vars['splice_change']=='yes')].reset_index(drop=True)

In [ ]:
# 106 variants with spliceAI score > 0.9
# 47 variants with significant effect in splicing, 18 variants no effect, 9 maybe effect
# 32 too low coverage, 19 no effect, 13 maybe effect
for_splai = make_float(new_onl_vars[new_onl_vars['SpliceAI_pred_DS_AG'].notnull()].copy())
len(for_splai[((for_splai['SpliceAI_pred_DS_AG']>0.9)|(for_splai['SpliceAI_pred_DS_AL']>0.9)|
               (for_splai['SpliceAI_pred_DS_DG']>0.9)|(for_splai['SpliceAI_pred_DS_DL']>0.9))&
              (for_splai['too_low_coverage']=='yes')&(for_splai['splice_change']=='no')
             ].groupby(by=cols_cpra))

In [ ]:
# 63 variants previously of uncertain significance or not in ClinVar with splice change and enough coverage
for_splai[['#CHROM', 'POS', 'REF', 'ALT', 'consensus_sequence_IGV_region_left', 
           'consensus_sequence_IGV_sequence_left', 'consensus_sequence_IGV_region_right',
           'consensus_sequence_IGV_sequence_right', 'INFO_IGV', 'too_low_coverage', 'splice_change', 
           'shortcut', 'HGVSc', 'HGVSg', 'HGVSp']][((for_splai['ClinVar_Pathogenicity']=='Uncertain Significance')|
                         (for_splai['ClinVar_Pathogenicity']=='uncertain_significance')|
                         (for_splai['ClinVar_Pathogenicity']=='Uncertain significance')|
                         (for_splai['ClinVar_Pathogenicity'].isnull()))&
                        (for_splai['too_low_coverage']=='no')&
                        (for_splai['splice_change']=='yes')]

In [ ]:
for_MES = make_float(vars_df[vars_df['MaxEntScan_diff']].notnull().copy())
for_MES_disr = for_MES[(for_MES['MaxEntScan_diff']>=0)&(for_MES['MaxEntScan_alt']<6.2)]
for_MES_new = for_MES[(['MaxEntScan_diff']<0)&(['MaxEntScan_alt']>8.5)]

#### Variants per prioritization criterion

In [ ]:
#vars_grpd_splai = vars_grpd[vars_grpd['SpliceAI_pred_DP_AG'].notnull()].copy()
#vars_grpd_splai = make_float(vars_grpd_splai)

#vars_grpd_splai = vars_grpd_splai[cols_cpra][((vars_grpd_splai['SpliceAI_pred_DS_AG']>0.9)|
                                              #(vars_grpd_splai['SpliceAI_pred_DS_AL']>0.9)|
                                              #(vars_grpd_splai['SpliceAI_pred_DS_DG']>0.9)|
                                              #(vars_grpd_splai['SpliceAI_pred_DS_DL']>0.9))
                                    #].drop_duplicates()
#vars_grpd_splai.reset_index(drop=True, inplace=True)

## Input

In [ ]:
df_xlsx = pd.read_excel('01_xlsx_files/2021-08-05_all_variants.xlsx')
all_vars = pd.read_pickle('00_dataframes/all_vars')

all_vars['#CHROM'] = pd.Categorical(all_vars['#CHROM'],
                                    categories=['1','2','3','4','5','6','7','8','9','10','11','12', '13', '14', 
                                                '15', '16', '17', '18', '19', '20', '21', '22', 'X', 'Y'],
                                    ordered=True)
all_vars = all_vars.sort_values(by=['#CHROM', 'POS', 'REF', 'ALT'], ignore_index=True)

cols_cpra = ['#CHROM', 'POS', 'REF', 'ALT']
vars_grpd = all_vars.drop_duplicates(subset=cols_cpra)

## Drop duplicates

In [ ]:
# duplicates for specific variants
#len(df_xlsx), len(df_xlsx.drop_duplicates(subset=cols_cpra)) #(328, 272)
grpd_xlsx = df_xlsx.groupby(by=cols_cpra)
grpd_lst = [(i,len(df)) for i,df in grpd_xlsx if len(df)>1]

In [ ]:
# to search in each group for best keep
# grpd_xlsx.get_group(grpd_lst[42][0])[['#CHROM', 'POS', 'REF', 'ALT', 'INFO_IGV','too_low_coverage','splice_change', 
                                     #'PIDs', 'consensus_sequence_IGV_region_left', 
                                     #'consensus_sequence_IGV_sequence_left','consensus_sequence_IGV_region_right',
                                     #'consensus_sequence_IGV_sequence_right']]

In [ ]:
for_drop = [7, 17, 9, 36, 78, 74, 90, 91, 89, 101, 104, 102, 105, 113, 122, 148, 152, 149, 150, 155, 157, 154, 
            166, 165, 183, 177, 174, 185, 202, 220, 211, 215, 205, 207, 229, 239, 235, 241, 250, 265, 267, 264, 
            263, 268, 271, 280, 284, 287, 289, 292, 304, 314, 316, 313, 325, 327]
df_xlsx.drop(index=for_drop, inplace=True)
df_xlsx.reset_index(drop=True, inplace=True)

In [ ]:
df_xlsx['shortcut'] = (df_xlsx['#CHROM'].astype(str) + '_' + df_xlsx['POS'].astype(str) + '_' + 
                       df_xlsx['REF'].astype(str) + '_' + df_xlsx['ALT'].astype(str))

In [ ]:
vars_df = pd.merge(df_xlsx[['#CHROM', 'POS', 'REF', 'ALT', 'consensus_sequence_IGV_region_left',
                               'consensus_sequence_IGV_sequence_left', 'consensus_sequence_IGV_region_right',
                               'consensus_sequence_IGV_sequence_right', 'INFO_IGV', 'too_low_coverage',
                               'splice_change', 'shortcut']], all_vars, on=cols_cpra, how='left')

In [ ]:
#xlsx_df(vars_df, 'variants_seen_no_dup')

## New excel files without duplicates

In [ ]:
splAI_high = vars_df[vars_df['SpliceAI_pred_DS_AG'].notnull()].copy()
splAI_high = make_float(splAI_high)
splAI_high = splAI_high[((splAI_high['SpliceAI_pred_DS_AG']>0.7)|
                         (splAI_high['SpliceAI_pred_DS_AL']>0.7)|
                         (splAI_high['SpliceAI_pred_DS_DG']>0.7)|
                         (splAI_high['SpliceAI_pred_DS_DL']>0.7))].drop_duplicates()

for i in ['consensus_sequence_IGV_region_left', 'consensus_sequence_IGV_sequence_left', 'too_low_coverage',
          'consensus_sequence_IGV_region_right', 'consensus_sequence_IGV_sequence_right', 'splice_change',
          'INFO_IGV']:
    splAI_high[i] = np.nan

xlsx_df(splAI_high, 'new_splAI_high')

In [ ]:
MES_high_disr = pd.read_excel('../04_annotations/03_prioritized_xlsx/new_MES_high_disr.xlsx')
MES_high_disr['from_where'] = 'MES_high_disr'
MES_high_new = pd.read_excel('../04_annotations/03_prioritized_xlsx/new_MES_high_new.xlsx')
MES_high_new['from_where'] = 'MES_high_new'
dbsc_high = pd.read_excel('../04_annotations/03_prioritized_xlsx/new_dbsc_high.xlsx')
dbsc_high['from_where'] = 'dbsc_high'
splAI_high_new = pd.read_excel('01_xlsx_files/new_splAI_high.xlsx')
splAI_high_new['from_where'] = 'splAI_high'

In [ ]:
all_for_new = MES_high_disr.append(MES_high_new, ignore_index=True)
all_for_new = all_for_new.append(dbsc_high, ignore_index=True)
all_for_new = all_for_new.append(splAI_high_new, ignore_index=True).drop_duplicates(subset=cols_cpra, 
                                                                                    ignore_index=True)
all_for_new['shortcut'] = (all_for_new['#CHROM'].astype(str) + '_' + all_for_new['POS'].astype(str) + '_' + 
                           all_for_new['REF'].astype(str) + '_' + all_for_new['ALT'].astype(str))

In [ ]:
all_for_new = all_for_new[~all_for_new['shortcut'].isin(vars_df['shortcut'])].reset_index(drop=True)
all_for_new.to_excel('01_xlsx_files/2021-08-06_rest_vars_nodup.xlsx')

## IGV splice effect seen

In [ ]:
# splice effect seen, safe (not too low coverage, only if sure (splice_change = yes)) --> 80 variants
spl_yes = vars_df[(vars_df['too_low_coverage']=='no')&(vars_df['splice_change']=='yes')]

In [ ]:
spl_yes_splai = spl_yes_df[spl_yes_df['SpliceAI_pred_DS_AG'].notnull()].copy()
make_float(spl_yes_splai)
spl_yes_splai_high = spl_yes_splai[((spl_yes_splai['SpliceAI_pred_DS_AG']>0.9)|
            (spl_yes_splai['SpliceAI_pred_DS_AL']>0.9)|
            (spl_yes_splai['SpliceAI_pred_DS_DG']>0.9)|
            (spl_yes_splai['SpliceAI_pred_DS_DL']>0.9))].copy()
len(spl_yes_splai), len(spl_yes_df), len(spl_yes_splai_high), len(spl_yes_splai_high[spl_yes_splai_high['Splice Project Gene Priority']=='Hot (ACMG / MASTER)'])

In [ ]:
# 106 variants with spliceAI score > 0.9
# 46 variants with significant effect in splicing, 17 variants no effect, 33 too low coverage, 10 maybe effect
for_splai = make_float(vars_df[vars_df['SpliceAI_pred_DS_AG'].notnull()].copy())
len(for_splai[((for_splai['SpliceAI_pred_DS_AG']>0.9)|(for_splai['SpliceAI_pred_DS_AL']>0.9)|
               (for_splai['SpliceAI_pred_DS_DG']>0.9)|(for_splai['SpliceAI_pred_DS_DL']>0.9))&
              (for_splai['too_low_coverage']=='no')&(for_splai['splice_change']=='maybe')].groupby(by=cols_cpra))

In [ ]:
for_MES = make_float(vars_df[vars_df['MaxEntScan_diff']].notnull().copy())
for_MES_disr = for_MES[(for_MES['MaxEntScan_diff']>=0)&(for_MES['MaxEntScan_alt']<6.2)]
for_MES_new = for_MES[(['MaxEntScan_diff']<0)&(['MaxEntScan_alt']>8.5)]

## Variants per prioritization criterion

In [ ]:
#vars_grpd_splai = vars_grpd[vars_grpd['SpliceAI_pred_DP_AG'].notnull()].copy()
#vars_grpd_splai = make_float(vars_grpd_splai)

#vars_grpd_splai = vars_grpd_splai[cols_cpra][((vars_grpd_splai['SpliceAI_pred_DS_AG']>0.9)|
                                              #(vars_grpd_splai['SpliceAI_pred_DS_AL']>0.9)|
                                              #(vars_grpd_splai['SpliceAI_pred_DS_DG']>0.9)|
                                              #(vars_grpd_splai['SpliceAI_pred_DS_DL']>0.9))
                                    #].drop_duplicates()
#vars_grpd_splai.reset_index(drop=True, inplace=True)